In [53]:
import pandas as pd
import os

papers = pd.read_csv(
    #"../rawData/sampler_10ktexts_perdecade.ALL2.tsv",
    "../rawData/sampler_50ktexts_perdecade.headed.ALL.tsv",
    sep='\t',
    names=["timePeriod", "index", "rawText"]
)
papers.drop(papers[~papers['timePeriod'].str.endswith('POS.rand')].index, inplace=True)


papers.head()
papers.info()

C:\Users\sandr\AppData\Local\Temp\ipykernel_18744\4148437997.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  papers = pd.read_csv(


<class 'pandas.core.frame.DataFrame'>
Index: 802378 entries, 1 to 802379
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   timePeriod  802378 non-null  object
 1   index       802378 non-null  object
 2   rawText     802378 non-null  object
dtypes: object(3)
memory usage: 24.5+ MB


In [54]:
papers['timePeriod'] = papers['timePeriod'].map(lambda x: x.rstrip('X.POS.rand'))
papers['timePeriod'] = papers['timePeriod'].astype(str) + '0'
papers.head()

,timePeriod,index,rawText
1,1800,1,I_PP scarcely_RB know_VBP how_WRB to_TO answer...
2,1800,2,It_PP had_VBD been_VBN too_RB much_JJ the_DT p...
3,1800,3,gent_NN ._SENT
4,1800,4,The_DT attention_NN of_IN the_DT public_NN was...
5,1800,5,"With_IN regard_NN to_TO the_DT Oath_NP ,_, the..."


In [55]:
papers['timePeriod'] = pd.to_datetime(papers['timePeriod'], format='%Y')
papers['timePeriod'] = pd.DatetimeIndex(papers['timePeriod']) #.year
papers.head()

,timePeriod,index,rawText
1,1800-01-01,1,I_PP scarcely_RB know_VBP how_WRB to_TO answer...
2,1800-01-01,2,It_PP had_VBD been_VBN too_RB much_JJ the_DT p...
3,1800-01-01,3,gent_NN ._SENT
4,1800-01-01,4,The_DT attention_NN of_IN the_DT public_NN was...
5,1800-01-01,5,"With_IN regard_NN to_TO the_DT Oath_NP ,_, the..."


In [56]:
#reducedPapers = papers[papers["timePeriod"].isin(pd.date_range("1800-01-01", "1830-01-01"))]
#reducedPapers = papers[papers["timePeriod"].isin(pd.date_range("1800-01-01", "1890-01-01"))]
#reducedPapers = papers[papers["timePeriod"].isin(pd.date_range("1800-01-01", "1820-01-01"))]

#reducedPapers = papers.groupby("timePeriod").sample(frac=0.5, random_state=42)
reducedPapers = papers
reducedPapers.tail()
#reducedPapers.tail()

,timePeriod,index,rawText
802375,2000-01-01,49996,I_PP beg_VBP to_TO move_VB ._SENT
802376,2000-01-01,49997,<p> Ms_NN Blears_NP :_: I_PP certainly_RB shal...
802377,2000-01-01,49998,<lb/> We_PP want_VBP to_TO make_VB improvement...
802378,2000-01-01,49999,<lb/> I_PP turn_VBP now_RB to_TO the_DT propos...
802379,2000-01-01,50000,<lb/> I_PP seek_VBP to_TO persuade_VB the_DT C...


In [57]:
import nltk
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words.update({
    'hon',
    'mr',
    'gentleman',
    'gent', 'gen',
    'right',
    'house',
    'opinion',
    'sir',
    'say',
    'friend',
    'member',
    'think',
    'go',
    'motion'
    'noble',
    'lord',
    'noble',
    'case',
    'year',
    'speak',
    'speech'
})

wn = WordNetLemmatizer()

def lemmatize(word):
    if '_' not in word: return ""
    token, posStart = word.split('_')[0].lower(), word.split('_')[1].upper()
    if posStart.startswith('N'):
        return wn.lemmatize(token, wordnet.NOUN)
    elif posStart.startswith('V'):
        return wn.lemmatize(token, wordnet.VERB)
    elif posStart.startswith('R'):
        return wn.lemmatize(token, wordnet.ADV)
    elif posStart.startswith('J'):
        return wn.lemmatize(token, wordnet.ADJ)
    return ""



def preprocess_text(text):
    tokens = simple_preprocess(text)
    clean = [lemmatize(word) for word in tokens if lemmatize(word)!=""]
    words = [word for word in clean if word.isalpha()]
    lemmas = [word for word in words if word not in stop_words]
    return " ".join(lemmas)

reducedPapers.loc[:, 'processedText'] = reducedPapers['rawText'].apply(preprocess_text).astype(str)
reducedPapers.loc[:, 'processedText'] = reducedPapers['processedText'].astype("str")
reducedPapers.dropna(subset = ['processedText'], inplace = True, how='any')
reducedPapers.reset_index(drop=True, inplace=True)
reducedPapers.info()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802378 entries, 0 to 802377
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   timePeriod     802378 non-null  datetime64[ns]
 1   index          802378 non-null  object        
 2   rawText        802378 non-null  object        
 3   processedText  802378 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.5+ MB


In [58]:

reducedPapers.to_csv('../processedData/processedData.csv', sep=',', columns=[ 'index','timePeriod', 'processedText'])

In [59]:
test = pd.read_csv(
    "../processedData/processedData.csv",
    sep=',',
    index_col=0
)
test.tail()
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 802378 entries, 0 to 802377
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   index          802378 non-null  int64 
 1   timePeriod     802378 non-null  object
 2   processedText  755621 non-null  object
dtypes: int64(1), object(2)
memory usage: 24.5+ MB
